First, import needed modules

In [110]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [111]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters


Import datasets

In [112]:
btw13_es = pd.read_csv(f'{path}/data/elections/BTW13_Erststimmen_Wahlbezirke.csv', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})
btw13_es.head()

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_23579/4294177193.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  btw13_es = pd.read_csv(f'{path}/data/elections/BTW13_Erststimmen_Wahlbezirke.csv', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Gemeinde': str})


,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,BIG,pro Deutschland,FREIE WÄHLER,PARTEI DER VERNUNFT,Die PARTEI,Bergpartei,BGD,DKP,NEIN,WGr/EB
0,1,01,0,01,0,000,0,001,0,1701,...,0,0,0,0,0,0,0,0,0,0
1,1,01,0,01,0,000,0,002,0,1567,...,0,0,0,0,0,0,0,0,0,0
2,1,01,0,01,0,000,0,003,0,1670,...,0,0,0,0,0,0,0,0,0,0
3,1,01,0,01,0,000,0,004,0,1592,...,0,0,0,0,0,0,0,0,0,0
4,1,01,0,01,0,000,0,005,0,1277,...,0,0,0,0,0,0,0,0,0,0


Create the AGS, it follows the scheme ll r kk ggg (Land, Regierunsbezirk, Kreis, Gemeinde) -> combine these columns

In [113]:
btw13_es['AGS'] = btw13_es['Land'] + btw13_es['Regierungsbezirk'] + btw13_es['Kreis'] + btw13_es['Gemeinde']
btw13_es.head()
#dimensions = 87927 rows, 49 columns


,Wahlkreis,Land,Regierungsbezirk,Kreis,Verbandsgemeinde,Gemeinde,Kennziffer Briefwahlzugehörigkeit,Wahlbezirk,Bezirksart,Wahlberechtigte (A),...,pro Deutschland,FREIE WÄHLER,PARTEI DER VERNUNFT,Die PARTEI,Bergpartei,BGD,DKP,NEIN,WGr/EB,AGS
0,1,01,0,01,0,000,0,001,0,1701,...,0,0,0,0,0,0,0,0,0,01001000
1,1,01,0,01,0,000,0,002,0,1567,...,0,0,0,0,0,0,0,0,0,01001000
2,1,01,0,01,0,000,0,003,0,1670,...,0,0,0,0,0,0,0,0,0,01001000
3,1,01,0,01,0,000,0,004,0,1592,...,0,0,0,0,0,0,0,0,0,01001000
4,1,01,0,01,0,000,0,005,0,1277,...,0,0,0,0,0,0,0,0,0,01001000


Clean dataset

In [114]:
btw13_es = btw13_es.groupby(['AGS']).sum() # collapse data along AGS column, sums votes over all constituencies in a municipality 11348 columns left
afd = btw13_es.pop('AfD') # remove column afd 
btw13_es['AfD']= afd # insert afd at the end
# list(data.columns)
andere = [ 'PIRATEN', 'NPD', 'Tierschutzpartei', 'REP', 'ÖDP', 'FAMILIE', 'Bündnis 21/RRP', 'RENTNER', 'BP', 'PBC', 'BüSo', 'DIE VIOLETTEN', 'MLPD',
 'Volksabstimmung', 'BIG', 'pro Deutschland', 'FREIE WÄHLER', 'PARTEI DER VERNUNFT', 'Die PARTEI', 'Bergpartei', 'BGD', 'DKP', 'NEIN', 'WGr/EB']
btw13_es['Andere'] = btw13_es[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw13_es['Union'] = btw13_es[union].sum(axis=1) # sum votes for 'Union' Parties
btw13_es = btw13_es[['Wahlberechtigte (A)', 'Wähler (B)', 'Union', 'SPD', 'FDP', 'DIE LINKE', 'GRÜNE', 'AfD', 'Andere']]
btw13_es.head()

,Wahlberechtigte (A),Wähler (B),Union,SPD,FDP,DIE LINKE,GRÜNE,AfD,Andere
AGS,,,,,,,,,
01001000,70321,46827,15647,18915,807,3250,4533,1459,1627
01002000,187464,132054,41409,57073,2703,7220,13464,3616,5045
01003000,167287,113351,39973,46127,2709,6205,8716,3737,4461
01004000,61095,39957,15857,15891,655,1768,1993,1707,1463
01051001,2695,1517,778,415,78,64,62,60,44


Merge with treatment dataset

In [117]:
treatment = pd.read_csv(f'{path}/data/treatment.csv', converters = {'AGS': str})
data = pd.merge(btw13_es, treatment, on='AGS', indicator=True)
data = data.set_index('AGS')
data = data.drop(columns=['Unnamed: 0', '_merge'])
data.rename(columns = {'Wahlberechtigte (A)':'Wahlberechtigte', 'Wähler (B)': 'Wähler'}, inplace = True)
data = data.reindex(columns=['GEN', 'Wahlberechtigte', 'Wähler', 'Union', 'SPD', 'FDP', 'DIE LINKE', 'GRÜNE', 'AfD', 'Andere', 'treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100'])
data.head()

,GEN,Wahlberechtigte,Wähler,Union,SPD,FDP,DIE LINKE,GRÜNE,AfD,Andere,treated_0,treated_10,treated_20,treated_30,treated_50,treated_100
AGS,,,,,,,,,,,,,,,,
01001000,Flensburg,70321,46827,15647,18915,807,3250,4533,1459,1627,0,0,0,0,0,0
01002000,Kiel,187464,132054,41409,57073,2703,7220,13464,3616,5045,0,0,0,0,0,0
01003000,Lübeck,167287,113351,39973,46127,2709,6205,8716,3737,4461,0,0,0,0,0,0
01004000,Neumünster,61095,39957,15857,15891,655,1768,1993,1707,1463,0,0,0,0,0,1
01051001,Albersdorf,2695,1517,778,415,78,64,62,60,44,0,0,0,0,1,1
